# 2d tie contact

Author: Jukka Aho

Abstract: 2d tie contact.

Model:


![model](http://s4.postimg.org/u9yqeryul/Screenshot_from_2015_11_24_02_00_00.png)

Each element is modelled as own "body" and they are connected using tie contacts. Segments 5-6 and 9-10 and 6-7 are slave surfaces, so node 6 or 9 is on at least two tie contacts as slave node. Moreover this model has dirichlet boundary $y=0$ at bottom of body 1 and $x=0$ on left. To get the accurate solution one needs to minimize 
\begin{equation}
\frac{15}{2}u_1^4 + 60 u_1^3 + \frac{15}{4}u_1^2 u_2^2 + 15 u_1^2 u_2 + 120 u_1^2 + 15 u_1 u_2^2 + 60 u_1 u_2 + \frac{15}{2}u_2^4 + 60 u_2^3 + 120 u_2^2 + 50 u_2,
\end{equation}
which gives approximate $u_1 = 0.0634862$ and $u_2 = -0.277183$ for the displacement of upper right corner.
[Wolfram](http://www.wolframalpha.com/input/?i=local+minimum+15*x^4%2F2+%2B+60*x^3+%2B+15*x^2*y^2%2F4+%2B+15*x^2*y+%2B+120*x^2+%2B+15*x*y^2+%2B+60*x*y+%2B+15*y^4%2F2+%2B+60*y^3+%2B+120*y^2+%2B+50*y).

In [1]:
using JuliaFEM
using JuliaFEM: Element, Seg2, Quad4, PlaneStressElasticityProblem, DirichletProblem, MortarProblem, DirectSolver

In [2]:
nodes = Dict{Int64, Vector{Float64}}(
    1 => [0.0, 0.0],
    2 => [2.0, 0.0],
    3 => [2.0, 1.0],
    4 => [0.0, 1.0],
    5 => [0.0, 1.0],
    6 => [1.0, 1.0],
    7 => [1.0, 2.0],
    8 => [0.0, 2.0],
    9 => [1.0, 1.0],
    10 => [2.0, 1.0],
    11 => [2.0, 2.0],
    12 => [1.0, 2.0]);

In [3]:
connectivity = Dict{Int64, Vector{Int64}}(
    1 => [1, 2, 3, 4],
    2 => [5, 6, 7, 8],
    3 => [9, 10, 11, 12]);

In [4]:
elements = Element[]
for c in values(connectivity)
    element = Quad4(c)
    element["geometry"] = Vector{Float64}[nodes[i] for i in c]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    push!(elements, element)
end
length(elements)

3

Create three bodies, each containing one element.

In [5]:
body1 = PlaneStressElasticityProblem()
body2 = PlaneStressElasticityProblem()
body3 = PlaneStressElasticityProblem()
push!(body1, elements[1])
push!(body2, elements[2])
push!(body3, elements[3]);

Surface traction to the top of bodies 2 and 3:

In [6]:
t2 = Seg2([8, 7])
t2["geometry"] = Vector{Float64}[nodes[8], nodes[7]]
t2["displacement traction force"] = Vector{Float64}[[0.0, -100.0], [0.0, -100.0]]
t3 = Seg2([12, 11])
t3["geometry"] = Vector{Float64}[nodes[12], nodes[11]]
t3["displacement traction force"] = Vector{Float64}[[0.0, -100.0], [0.0, -100.0]]
push!(body2, t2)
push!(body3, t3);

Boundary conditions: $x=0$ for left boundary.

In [7]:
dx1 = Seg2([1, 4])
dx1["geometry"] = Vector[nodes[1], nodes[4]]
dx1["displacement 1"] = 0.0
dx2 = Seg2([5, 8])
dx2["geometry"] = Vector[nodes[5], nodes[8]]
dx2["displacement 1"] = 0.0
bc1 = DirichletProblem("displacement", 2)
push!(bc1, dx1)
push!(bc1, dx2);

$y=0$ for bottom of model

In [8]:
dy1 = Seg2([1, 2])
dy1["geometry"] = Vector[nodes[1], nodes[2]]
dy1["displacement 2"] = 0.0
bc2 = DirichletProblem("displacement", 2)
push!(bc2, dy1);

Mortar boundary conditions: tie contact between body 1 and body 2

In [9]:
rotation_matrix(phi) = [cos(phi) -sin(phi); sin(phi) cos(phi)]

master1 = Seg2([4, 3])
master1["geometry"] = Vector[nodes[4], nodes[3]]
slave1 = Seg2([5, 6])
slave1["geometry"] = Vector[nodes[5], nodes[6]]
slave1["master elements"] = Element[master1]
slave1["nodal ntsys"] = Matrix[rotation_matrix(-pi/2), rotation_matrix(-pi/2)]
contact1 = MortarProblem("displacement", 2)
push!(contact1, slave1);

Tie contact between body 1 and body 3

In [10]:
slave2 = Seg2([9, 10])
slave2["geometry"] = Vector[nodes[9], nodes[10]]
slave2["nodal ntsys"] = Matrix[rotation_matrix(-pi/2), rotation_matrix(-pi/2)]
slave2["master elements"] = Element[master1]
contact2 = MortarProblem("displacement", 2)
push!(contact2, slave2);

Tie contact between body 2 and body 3

In [11]:
master2 = Seg2([6, 7])
master2["geometry"] = Vector[nodes[6], nodes[7]]
slave3 = Seg2([9, 12])
slave3["geometry"] = Vector[nodes[9], nodes[12]]
slave3["nodal ntsys"] = Matrix[rotation_matrix(0.0), rotation_matrix(0.0)]
slave3["master elements"] = Element[master2]
contact3 = MortarProblem("displacement", 2)
push!(contact3, slave3);

All defined. Solve it.

In [12]:
solver = DirectSolver()
push!(solver, body1)
push!(solver, body2)
push!(solver, body3)
push!(solver, bc1)
push!(solver, bc2)
push!(solver, contact1)
push!(solver, contact2)
push!(solver, contact3);

In [13]:
iterations, converged = call(solver, 0.0)

INFO: # of field problems: 3
INFO: # of boundary problems: 5
INFO: Starting iteration 1
INFO: # of dofs: 24, # of interface dofs: 15
INFO: solved. length of solution vector = 48
INFO: Iteration took 9.311098465 seconds


(5,true)

INFO: Starting iteration 2
INFO: # of dofs: 24, # of interface dofs: 15
INFO: solved. length of solution vector = 48
INFO: Iteration took 0.003787437 seconds
INFO: Starting iteration 3
INFO: # of dofs: 24, # of interface dofs: 15
INFO: solved. length of solution vector = 48
INFO: Iteration took 0.020931551 seconds
INFO: Starting iteration 4
INFO: # of dofs: 24, # of interface dofs: 15
INFO: solved. length of solution vector = 48
INFO: Iteration took 0.003763852 seconds
INFO: Starting iteration 5
INFO: # of dofs: 24, # of interface dofs: 15
INFO: solved. length of solution vector = 48
INFO: Iteration took 0.003679408 seconds


In [14]:
using JuliaFEM.Test

@test converged

X = elements[2]("geometry", [1.0, 1.0], 0.0)
u = elements[2]("displacement", [1.0, 1.0], 0.0)
info("displacement at $X = $u")
@test isapprox(u, [0.0634862, -0.277183], atol=1.0e-5)

INFO: displacement at [2.0,2.0] = [0.06348623177789343,-0.27718303785565257]
